In [11]:
import pandas as pd
from collections import defaultdict
from pandas import Series, DataFrame

In [5]:
entity_data = pd.read_json('./project.json')

In [7]:
del entity_data['file_upload'], entity_data['predictions'], entity_data['drafts'], entity_data['meta'], entity_data['created_at'], entity_data['updated_at'], entity_data['id']

In [8]:
origin = ['no_relation', 'anm:서식지', 'anm:대체표현', 'anm:신체적 특징', 'anm:비신체적 특징', 'anm:사냥감', 'anm:역할', 'anm:유래', 'anm:개체수', 'org:하위집단']
change = ['no_relation', 'anm:habitat', 'anm:alternate_name','anm:physical', 'anm:habit', 'anm:enemy', 'anm:role', 'anm:origin', 'anm:number_of_members', 'org:sub_group']


dict_change = dict(zip(origin, change))

In [10]:
SUB = []
OBJ = []
sentences = []
relations = []

for annotations, data in zip(entity_data['annotations'], entity_data['data']):
    key_dict = defaultdict(str)
    list = annotations[0]['result']
    sentence = data['text']
    
    for elem in list:
        if 'value' in elem:
            key_dict[elem['id']] = elem['value']
        
        else:
            from_id = key_dict[elem['from_id']]
            to_id = key_dict[elem['to_id']]
            
            from_id['type'] = str(from_id['labels'][0]).split('_')[1].lower()            
            
            if elem['direction'] == 'right':
                SUB.append(from_id)
                OBJ.append(to_id)
                sentences.append(sentence)
                relations.append(dict_change[elem['labels'][0]])
            else:
                OBJ.append(from_id)
                SUB.append(to_id)
                sentences.append(sentence)
                relations.append(dict_change[elem['labels'][0]])
    
    del(key_dict)

In [12]:
df = pd.DataFrame({'sentence':sentences, 'subject_entity':SUB, 'object_entity':OBJ, 'class':relations})

In [13]:
df.head()

,sentence,subject_entity,object_entity,class
0,"중국산악고양이는 야행성으로, 주로 쥐, 새, 두더지, 토끼 등을 먹는다.","{'start': 0, 'end': 7, 'text': '중국산악고양이', 'lab...","{'start': 25, 'end': 28, 'text': '두더지', 'label...",anm:enemy
1,"이 고양이는 2002년 이래로 IUCN 적색 목록에 취약종으로 등재되어 있고, 현재...","{'start': 16, 'end': 21, 'text': ' IUCN', 'lab...","{'start': 2, 'end': 5, 'text': '고양이', 'labels'...",org:sub_group
2,"이 고양이는 2002년 이래로 IUCN 적색 목록에 취약종으로 등재되어 있고, 현재...","{'start': 2, 'end': 5, 'text': '고양이', 'labels'...","{'start': 51, 'end': 59, 'text': '10,000마리', '...",anm:number_of_members
3,중국산악고양이는 고양이속에 속하는 소형 고양이과 동물의 일종이다.,"{'start': 19, 'end': 26, 'text': '소형 고양이과', 'l...","{'start': 0, 'end': 7, 'text': '중국산악고양이', 'lab...",org:sub_group
4,"""제네타""(""Genetta"")라는 학명은 1817년 조르주 퀴비에(Georges C...","{'start': 1, 'end': 4, 'text': '제네타', 'labels'...","{'start': 29, 'end': 36, 'text': '조르주 퀴비에', 'l...",anm:origin


In [14]:
for data, data2 in zip(df['object_entity'],df['subject_entity']):
     if not 'type' in data:
            data['type'] = data['labels'][0].split('_')[1].lower()   
     if not 'type' in data2:
            data2['type'] = data2['labels'][0].split('_')[1].lower()   

In [15]:
for sentence, data in zip(df['sentence'],df['subject_entity']):
     if not 'type' in data:
            print(sentence)

In [16]:
for data, data2 in zip(df['object_entity'],df['subject_entity']):
    if 'labels' in data:
        data.pop('labels')
    if 'labels' in data2:
        data2.pop('labels')

In [17]:
def special_token_sentence(text, subj_start, subj_end, obj_start, obj_end):
    sentence1: str
    sentence2: str

    if subj_start < obj_start:
        sentence1 = text[:obj_start] + '<obj:' + text[obj_start:obj_end] + '>' + text[obj_end:]
        new_text = sentence1[:subj_start] + '<sbj:' + sentence1[subj_start:subj_end] + '>' + sentence1[subj_end:]

    else:
        sentence1 = text[:subj_start] + '<sbj:' + text[subj_start:subj_end] + '>' + text[subj_end:]
        new_text = sentence1[:obj_start] + '<obj:' + sentence1[obj_start:obj_end] + '>' + sentence1[obj_end:]
    return new_text

In [18]:
new_sentence = []
for sentence, sbj, obj in zip(df['sentence'], df['subject_entity'], df['object_entity']):
    new_sentence.append(special_token_sentence(sentence, sbj['start'], sbj['end'], obj['start'], obj['end']))

In [19]:
df['new_sentence'] = new_sentence

In [20]:
df = df[['sentence', 'new_sentence', 'subject_entity', 'object_entity', 'class']]

In [ ]:
df.to_csv('./file.csv', index=False)